#  Leitor de PDF 

O seguinte Modelo tem com objetivo realizar leitura de arquivos com extensão ".pdf"

Para os arquivos pdf de origem utiliza-se a biblioteca pdfplumber; já os arquivos scaneados
em formato pdf são lido mediante a biblioteca pytsseract(OCR).

Esse modelo é um pequeno exemplo de um modelo real que desenvolvi para um projeto,
o qual realiza a leitura do pdf e entrega um arquivo xml. Logos as nota fiscais de
serviço  são lidas e transformadas em xml e entregues para leitura do ERP.

Após a coseguir realizar a leitura do arquivo podemos realizar manipulação
dos dados que serão entregues no tipo "str". Sendo assim, podemos utilizar
a biblioteca re(Expressões Regulares).
Utilizarei com exemplo duas notas fiscais fictícias, as quais estão em pdf.
A primeira é um pdf scaneado e a segunda é um arquivo de origem.


In [1]:
# imports
import pdfplumber
import os
from pdf2image import convert_from_path
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'Tesseract-OCR\\tesseract.exe'
import cv2
import re

## Algoritimo de Leitura

In [2]:
# Metódo para lista os itens do diretório de leitura
def aqr_dir(url):
    dir_ = url
    path = [os.path.join(dir_, nome) for nome in os.listdir(dir_)]
    url_res = [arq for arq in path if os.path.isfile(arq)]
    arq = [nome.replace('.pdf',"") for nome in os.listdir(dir_)]
    
    return url_res, arq


# Metódo para realizar leitura de arquivo de origem pdf
def pdf(url):
    with pdfplumber.open(url) as pdf:        
        page =[pdf.pages[i] for i in range(len(pdf.pages))]      
        extracText = [page[i].extract_text() for i in range(len(page))]
                  
        return extracText
    

# Metódo para realizar leitura de arquivos de formatos img ou pdf scaneados
def ocr(url, arq):
        
    text = []
    res = []        
            
    def reduce(img):  
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        th, threshed = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (80,80))
        morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)
        cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        cnt = sorted(cnts, key=cv2.contourArea)[-1]
        x,y,w,h = cv2.boundingRect(cnt)
        dst = img[y:y+h, x:x+w]
        return dst
    
    img = convert_from_path(url, dpi=300, poppler_path=r'poppler-0.68.0\\bin')   
    im = [imgs.save('nf_img\\'+str(arq+'.png'),"PNG") for imgs in img]
                
    nfdir = 'nf_img'
    path = [os.path.join(nfdir, nome) for nome in os.listdir(nfdir)]
    url = [arq for arq in path if os.path.isfile(arq)]
    url_img = [u for u in url if ".png" in u]
                
    imagem = cv2.imread(url_img[0])
    img_res = reduce(imagem)
                
    gray = cv2.cvtColor(img_res, cv2.COLOR_BGR2GRAY)
    adp = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,85,80)        
    tx = (pytesseract.image_to_string(adp, lang='por',config='--psm 6'))
                
    [os.remove(u) for u in url_img if ".png" in u]
                
    return tx   

In [3]:
# Chamada ao metódo de leitura de diretórios
url, name = aqr_dir('nf')

In [4]:
#print dos itens
for u in url:
    print(u)

nf\nota_01.pdf
nf\nota_02.pdf


In [5]:
#print do nome dositens
for n in name:
    print(n)

nota_01
nota_02


In [6]:
# Leitura dos itens
tx = [p if p!= None else ocr(url[i],name[i]) for i in range(len(url)) for p in pdf(url[i])]

In [7]:
# Texto extraido da primeira nota fiscal (arqivo lido pelo ocr)
nota1 = tx[0].split('\n')[0:8] 
nota1

['Vu PREFEITURA DA CIDADE DO RIO DEJANEIRO — |É |Memen SA noa',
 'M ” DOOOO033',
 '= " SECRETARIA MUNICIPAL DE FAZENDA Data e Hora de Emissão',
 'ç Z A D4/02/2021 09:34:33',
 "% ' NOTA FISCAL DE SERVIÇOS ELETRONICA - NFS-e d o",
 'oA TS o ódigo de Verificação',
 '2O210217U34126214000120 - NOTA CARIOCA - RH2B-ENJT',
 'PRESTADOR DE SERVIÇOS']

In [8]:
# Texto extraido da segunda nota fiscal (arqivo lido pelo  pdfplumber)
nota2 = tx[1].split('\n')[0:8] 
nota2

['PREFEITURA MUNICIPAL DE RIO CLARO',
 'Número da',
 'SECRETARIA MUNICIPAL DE ECONOMIA E NFS-e',
 'NOTA FISCAL ELETRÔNICA DE SERVIÇO - NFS-e 107',
 'Código de Verificação',
 'Data e Hora da Emissão 04/02/2021 11:16:11 Competência 4/2/2021 M5R5YHNQB',
 'No. da NFS-e substituída',
 'Número do RPS Local da Prestação RIO CLARO - SP']

## Captura de dados com Expressões Regulares

In [9]:
# Captura da data de emissão da segunda nota
data = [r for t in nota2 for r in re.findall(r'[\d{2}/]*\d{4}',t) if r][0]
data

'04/02/2021'

Assim com o exemplo assim podemos extrair diversar iformações da nota fiscal
e depois entregar esses dados em um formato XML, por exemplo.
